In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# Define the number of classes
num_classes = 3

In [3]:
# Load the MobileNetV2 base model (without the top classification layer)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [4]:
# Custom layers on the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and preprocess the training data
train_generator = train_datagen.flow_from_directory(
    '../datasets/meme_dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 265 images belonging to 3 classes.


In [7]:
# Train the model
model.fit(train_generator, steps_per_epoch=train_generator.samples // train_generator.batch_size, epochs=10)

Epoch 1/10
8/8 [==============================] - 10s 685ms/step - loss: 0.2080 - accuracy: 0.8945
Epoch 2/10
8/8 [==============================] - 5s 637ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 3/10
8/8 [==============================] - 5s 626ms/step - loss: 0.0051 - accuracy: 0.9957
Epoch 4/10
8/8 [==============================] - 5s 624ms/step - loss: 7.9062e-06 - accuracy: 1.0000
Epoch 5/10
8/8 [==============================] - 6s 700ms/step - loss: 5.6412e-05 - accuracy: 1.0000
Epoch 6/10
8/8 [==============================] - 5s 621ms/step - loss: 6.3661e-04 - accuracy: 1.0000
Epoch 7/10
8/8 [==============================] - 6s 838ms/step - loss: 4.6066e-05 - accuracy: 1.0000
Epoch 8/10
8/8 [==============================] - 6s 802ms/step - loss: 2.3840e-04 - accuracy: 1.0000
Epoch 9/10
8/8 [==============================] - 6s 708ms/step - loss: 3.7202e-05 - accuracy: 1.0000
Epoch 10/10
8/8 [==============================] - 7s 797ms/step - loss: 6.2816e-07 - accurac

In [8]:
# Save the trained model
model.save('mobilenet_meme.h5')